# Redesign of nations and how they are created. 

Events are happening all the time. Several events happen before the Character quest begins. 

In [1]:
import pandas as pd
import numpy as np
import os

import os
import sys
import django

import altair as alt

sys.path.append("../..")
# Loading my project settings from prodweb. This allows me to load and query models.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "prodweb.settings")
django.setup()

# I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys, pickle

sys.path.append("../")
from lib.builders import people, towns, world as w
import lib.drawMaps as draw

pd.set_option("display.max_columns", 10)
pd.set_option("display.max_rows", 10)
%load_ext lab_black

In [2]:
with (open("../pickles/billmanhworld.pkl", "rb")) as pickle_file:
    world = pickle.load(pickle_file)

## This will be the new Nations builder module. 

In [3]:
from sklearn.cluster import KMeans


def cluster_nations(world):
    cities = world.df_features[world.df_features["terrain"] == "town"]
    world.nations = []
    world.df_features["nation number"] = np.nan
    world.df_features["nation"] = np.nan
    #     world.df_features["nation number"] = world.df_features["nation number"].
    k = KMeans(init="k-means++", n_clusters=world.culture.n_nations, n_init=10).fit(
        cities[["x", "y"]]
    )
    world.nations_k = k
    predict_nations(world)
    world.nations = [
        Nation(world, cluster=True, k=i) for i in np.unique(world.nations_k.labels_)
    ]
    return k


def predict_nations(world):
    world.df_features["nation number"] = world.nations_k.predict(
        world.df_features[["x", "y"]]
    )


class Nation:
    def __init__(self, world, **kwargs):
        self.name = self.name_nation(world)
        if kwargs.get("cluster", None):
            # Kmeans is the default (when the world is created)
            # Requires the integer value used when creating the world.
            self.cast_nation(world, kwargs.get("k"))
        self.diplomacy = {}
        self.capitol = None

    def cast_nation(self, world, k):
        world.df_features.loc[
            world.df_features["nation number"] == k, "nation"
        ] = self.name

    def name_nation(self, world):
        nation_name = world.culture.townNameGenerator()
        return nation_name

    def init_ruler(self, world):
        Person(world)

    def __repr__(self):
        return f"Nation of {self.name}"

    def set_capitol(self, world):
        ts = self.get_all_towns(world)
        cap = [
            t for t in ts if len(t.population) == max([len(t.population) for t in ts])
        ][0]
        cap.type = "capitol"
        self.capitol = cap

    def get_all_towns(self, world):
        all_towns = world.df_features.loc[
            (world.df_features["nation"] == self.name)
            & (world.df_features["terrain"] == "town"),
            "feature",
        ].unique()
        return [t for t in world.towns if t.name in all_towns]

    def get_random_town(self):
        return np.random.choice(self.towns)

    def getRuler(self):
        if self.ruler:
            return self.ruler
        else:
            return None

    def get_deplomacy_df(self):
        d = pd.DataFrame(self.diplomacy).T.reset_index(drop=False)
        d["nation"] = self.name
        d.columns = ["neighbor", "favor", "stance", "nation"]
        return d[["nation", "neighbor", "favor", "stance"]]

    def appointRuler(self, person):
        t = self.getCapitol_str(self.towns)
        self.ruler = person(f"ruler of {self.name}", t)


# nation = Nation(world,k=0)
# print(nation)

In [4]:
k = cluster_nations(world)
k

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [5]:
n = world.nations[1]

In [6]:
n.name

'Ragefort'

In [7]:
ts = n.get_all_towns(world)
ts

[town of Snowchild: population: 4 location: [13,33] founded 1,
 town of Servantsmore: population: 3 location: [13,31] founded 3,
 town of Doomrun: population: 1 location: [16,35] founded 8]

In [8]:
[t for t in ts if len(t.population) == max([len(t.population) for t in ts])][0]

town of Snowchild: population: 4 location: [13,33] founded 1

In [9]:
len(np.unique(world.df_features["nation"].dropna()))

8

In [10]:
n.capitol

In [11]:
n.set_capitol(world)

In [12]:
n.capitol

capitol of Snowchild: population: 4 location: [13,33] founded 1

In [13]:
world.towns

[town of Lionhaven: population: 4 location: [21,35] founded 1,
 capitol of Snowchild: population: 4 location: [13,33] founded 1,
 town of Servantsmore: population: 3 location: [13,31] founded 3,
 town of Heirford: population: 3 location: [21,36] founded 3,
 town of Stonefort: population: 4 location: [6,36] founded 3,
 town of Leafspring: population: 3 location: [0,30] founded 4,
 town of Badgerford: population: 2 location: [10,39] founded 4,
 town of Waterbridge: population: 3 location: [4,9] founded 5,
 town of Hearthail: population: 3 location: [3,14] founded 6,
 town of Doommain: population: 2 location: [23,39] founded 6,
 town of Icehost: population: 4 location: [39,19] founded 6,
 town of Kingshollow: population: 2 location: [32,17] founded 7,
 town of Demonsmoral: population: 2 location: [3,12] founded 7,
 town of Doomhaven: population: 1 location: [21,32] founded 8,
 town of Doomrun: population: 1 location: [16,35] founded 8,
 town of Flamerise: population: 2 location: [1,30] fo